In [12]:
# 数据一
import pandas as pd
df_transaction = pd.read_csv('ndm_transaction_list_all_20230705.csv')

/var/folders/gj/bwfyx1ks0p17lbgn6fl1nx1c0000gp/T/ipykernel_3262/1232885182.py:3: DtypeWarning: Columns (69,83,110,121,122,123,124,125,128) have mixed types. Specify dtype option on import or set low_memory=False.
  df_transaction = pd.read_csv('ndm_transaction_list_all_20230705.csv')


In [13]:
transaction_header_string = "dl_cd,dl_tm,bond_cd,net_prc,nmnl_vol,byr_instn_cn_full_nm,slr_instn_cn_full_nm,byr_instn_cd,slr_instn_cd"
header_string = "dl_cd,txn_dt,dl_tm,bsns_tm,bond_cd,bnds_nm,net_prc,yld_to_mrty,nmnl_vol,amnt,acrd_intrst,totl_acrd_intrst,all_prc,stlmnt_amnt,stlmnt_dt,ttm_yrly,byr_qt_cd,byr_instn_cd,byr_cfets_instn_cd,byr_instn_cn_full_nm,byr_instn_cn_shrt_nm,byr_instn_en_shrt_nm,byr_trdr_nm,byr_adrs,byr_trdr_fax,byr_lgl_rprsntv,byr_trdr_tel,buy_side_trdng_acnt_cd,byr_cptl_bnk_nm,byr_cptl_acnt_no,byr_pymnt_sys_cd,byr_dpst_acnt_nm,buy_side_dpst_cd,byr_trd_acnt_cfets_cd,byr_trd_acnt_cn_full_nm,byr_trd_acnt_cn_shrt_nm,byr_trd_acnt_en_shrt_nm,byr_cptl_acnt_nm,byr_trd_acnt_en_full_nm,slr_qt_cd,slr_instn_cd,slr_cfets_instn_cd,slr_instn_cn_full_nm,slr_instn_cn_shrt_nm,slr_instn_en_shrt_nm,slr_trdr_cd,slr_trdr_nm,slr_adrs,slr_trdr_fax,slr_lgl_rprsntv,slr_trdr_tel,sell_side_trdng_acnt_cd,slr_cptl_bnk_nm,slr_cptl_acnt_no,slr_pymnt_sys_cd,slr_dpst_acnt_nm,sell_side_dpst_acnt,slr_trd_acnt_cfets_cd,slr_trd_acnt_cn_full_nm,slr_trd_acnt_cn_shrt_nm,slr_trd_acnt_en_shrt_nm,slr_cptl_acnt_nm,slr_trd_acnt_en_full_nm,crt_tm,upd_tm"
transaction_header_string_list = transaction_header_string.split(',')
header_list = header_string.split(',')

In [14]:
def dataframe_to_json(df, column_mapping):
    """
    将 DataFrame 输出为 JSON 并修改列名。

    参数:
    df: pandas.DataFrame, 要转换的 DataFrame
    column_mapping: dict, 列名的映射，键为原列名，值为新列名

    返回:
    str: JSON 格式的字符串
    """
    df.rename(columns=column_mapping, inplace=True)
    return df.to_json(orient='records', force_ascii=False)

transaction_header_string = "dl_cd,dl_tm,bond_cd,net_prc,nmnl_vol,byr_instn_cn_full_nm,slr_instn_cn_full_nm,byr_instn_cd,slr_instn_cd"
transaction_header_string_list = transaction_header_string.split(',')

df = df_transaction[transaction_header_string_list][0:10]
df['nmnl_vol'] = df['nmnl_vol']/10000000

transaction_column_mapping = {
    'bond_cd': 'bond_cd',
    'dl_cd':'transactionId',
    'net_prc':'netPrice',
    'nmnl_vol':'transactionVolume',
     'dl_tm':'timeStamp',
}

json_result = dataframe_to_json(df, transaction_column_mapping)
print(json_result)


[{"transactionId":"CBT20230705312184","timeStamp":"2023-07-05 17:09:12+08:00","bond_cd":2305528,"netPrice":97.8675,"transactionVolume":9.0,"byr_instn_cn_full_nm":"交通银行股份有限公司","slr_instn_cn_full_nm":"中信银行股份有限公司","byr_instn_cd":100005,"slr_instn_cd":100007},{"transactionId":"CBT20230705312312","timeStamp":"2023-07-05 17:43:48+08:00","bond_cd":102381121,"netPrice":100.1792,"transactionVolume":1.0,"byr_instn_cn_full_nm":"中信证券股份有限公司","slr_instn_cn_full_nm":"中国民生银行股份有限公司","byr_instn_cd":100035,"slr_instn_cd":100015},{"transactionId":"CBT20230705312197","timeStamp":"2023-07-05 17:10:54+08:00","bond_cd":2305622,"netPrice":98.4568,"transactionVolume":8.0,"byr_instn_cn_full_nm":"华金证券股份有限公司","slr_instn_cn_full_nm":"中信银行股份有限公司","byr_instn_cd":302523,"slr_instn_cd":100007},{"transactionId":"CBT20230705312274","timeStamp":"2023-07-05 17:25:23+08:00","bond_cd":2228009,"netPrice":100.4087,"transactionVolume":2.0,"byr_instn_cn_full_nm":"平安基金管理有限公司","slr_instn_cn_full_nm":"山西证券股份有限公司","byr_instn_cd":400

In [16]:
from flask import Flask, g, jsonify
from flask import request
from flask_cors import CORS
from flask_pymongo import PyMongo
from flask_caching import Cache
import traceback
import json
from pydash import _
import numpy as np
import pandas as pd
import json
import math


In [23]:
def load_data():
    """
    Load data from the CSV files.

    Returns:
    tuple: Tuple containing DataFrames for MarketPrice, transaction, and instn_base_info.
    """
    # Load the data from the CSV files

    MarketPrice_header_list = ['bond_cd','trd_vol','txn_dt','mkt_data_upd_tm','term_to_expr_of_mrty_yrly', 'ltst_net_prc', 'opng_net_prc',
       'hghst_net_prc', 'lwst_net_prc', 'clsng_net_prc', 'net_prc_chng_ratio',
       'wghtd_avg_net_prc', 'prvs_opng_net_prc', 'prvs_clsng_net_prc',
       'prvs_wghtd_avg_net_prc']
    df_MarketPrice = pd.read_csv('mkt_price_list_all_20230705.csv')[MarketPrice_header_list]
    # Transaction_header_list = ['bond_cd','transactionId','netPrice','transactionVolume','date', 'timeStamp', 'byr_instn_cn_full_nm',
    #    'slr_instn_cn_full_nm', 'byr_instn_cd', 'slr_instn_cd']

    # header_string = "dl_cd,txn_dt,dl_tm,bsns_tm,bond_cd,bnds_nm,net_prc,yld_to_mrty,nmnl_vol,amnt,acrd_intrst,totl_acrd_intrst,all_prc,stlmnt_amnt,stlmnt_dt,ttm_yrly,byr_qt_cd,byr_instn_cd,byr_cfets_instn_cd,byr_instn_cn_full_nm,byr_instn_cn_shrt_nm,byr_instn_en_shrt_nm,byr_trdr_nm,byr_adrs,byr_trdr_fax,byr_lgl_rprsntv,byr_trdr_tel,buy_side_trdng_acnt_cd,byr_cptl_bnk_nm,byr_cptl_acnt_no,byr_pymnt_sys_cd,byr_dpst_acnt_nm,buy_side_dpst_cd,byr_trd_acnt_cfets_cd,byr_trd_acnt_cn_full_nm,byr_trd_acnt_cn_shrt_nm,byr_trd_acnt_en_shrt_nm,byr_cptl_acnt_nm,byr_trd_acnt_en_full_nm,slr_qt_cd,slr_instn_cd,slr_cfets_instn_cd,slr_instn_cn_full_nm,slr_instn_cn_shrt_nm,slr_instn_en_shrt_nm,slr_trdr_cd,slr_trdr_nm,slr_adrs,slr_trdr_fax,slr_lgl_rprsntv,slr_trdr_tel,sell_side_trdng_acnt_cd,slr_cptl_bnk_nm,slr_cptl_acnt_no,slr_pymnt_sys_cd,slr_dpst_acnt_nm,sell_side_dpst_acnt,slr_trd_acnt_cfets_cd,slr_trd_acnt_cn_full_nm,slr_trd_acnt_cn_shrt_nm,slr_trd_acnt_en_shrt_nm,slr_cptl_acnt_nm,slr_trd_acnt_en_full_nm,crt_tm,upd_tm"
    # header_list = header_string.split(',')
    # df_chain_data = pd.read_csv('server/static/chain_data/bond_2005496_2006_2402.csv')[header_list]

    transaction_header_string = "dl_cd,dl_tm,bond_cd,net_prc,nmnl_vol,byr_instn_cn_full_nm,slr_instn_cn_full_nm,byr_instn_cd,slr_instn_cd"
    transaction_header_string_list = transaction_header_string.split(',')
    df_transaction = pd.read_csv('ndm_transaction_list_all_20230705.csv')[transaction_header_string_list]

    # Select desired columns from instn_base_info DataFrame
    instn_base_info = pd.read_csv('20231030/rmb_hstry_actv_instn_base_info.csv')
    selected_columns = ['instn_cd', 'instn_tp', 'instn_cn_full_nm']
    result_df = instn_base_info[selected_columns]

    # Convert DataFrame to dictionary
    instn_dict = result_df.set_index('instn_cd').to_dict(orient='index')

    # Replace nan with None before converting to JSON
    for key, value in instn_dict.items():
        for sub_key, sub_value in value.items():
            if isinstance(sub_value, float) and math.isnan(sub_value):
                instn_dict[key][sub_key] = None

    return df_MarketPrice, df_transaction, instn_base_info, _, instn_dict

In [24]:
mktPrice, transaction, instn_base_info, df_chain_data, instn_dict = load_data()

/var/folders/gj/bwfyx1ks0p17lbgn6fl1nx1c0000gp/T/ipykernel_3262/1301511970.py:24: DtypeWarning: Columns (69,83,110,121,122,123,124,125,128) have mixed types. Specify dtype option on import or set low_memory=False.
  df_transaction = pd.read_csv('ndm_transaction_list_all_20230705.csv')[transaction_header_string_list]


In [27]:
transaction_header_string = "dl_cd,dl_tm,bond_cd,net_prc,nmnl_vol,byr_instn_cn_full_nm,slr_instn_cn_full_nm,byr_instn_cd,slr_instn_cd"
transaction_header_string_list = transaction_header_string.split(',')

dtype_spec = {
    'bond_cd': str,
    'transactionId': str,
    'netPrice': float,
    'transactionVolume': float,
    'timeStamp': str,
    'byr_instn_cn_full_nm': str,
    'slr_instn_cn_full_nm': str,
    'byr_instn_cd': str,
    'slr_instn_cd': str
}
df_transaction = pd.read_csv('ndm_transaction_list_all_20230705.csv',
                              usecols=transaction_header_string_list,
                              dtype=dtype_spec,
                              low_memory=False)


transaction_column_mapping = {
    'bond_cd': 'bond_cd',
    'dl_cd':'transactionId',
    'net_prc':'netPrice',
    'nmnl_vol':'transactionVolume',
     'dl_tm':'timeStamp',
}

df_transaction.rename(columns=transaction_column_mapping, inplace=True)

In [28]:
df_transaction

,transactionId,timeStamp,bond_cd,netPrice,transactionVolume,byr_instn_cd,byr_instn_cn_full_nm,slr_instn_cd,slr_instn_cn_full_nm
0,CBT20230705312184,2023-07-05 17:09:12+08:00,2305528,97.8675,90000000.0,100005,交通银行股份有限公司,100007,中信银行股份有限公司
1,CBT20230705312312,2023-07-05 17:43:48+08:00,102381121,100.1792,10000000.0,100035,中信证券股份有限公司,100015,中国民生银行股份有限公司
2,CBT20230705312197,2023-07-05 17:10:54+08:00,2305622,98.4568,80000000.0,302523,华金证券股份有限公司,100007,中信银行股份有限公司
3,CBT20230705312274,2023-07-05 17:25:23+08:00,2228009,100.4087,20000000.0,400511,平安基金管理有限公司,101169,山西证券股份有限公司
4,CBT20230705312263,2023-07-05 17:21:49+08:00,2271766,100.0329,120000000.0,101169,山西证券股份有限公司,100004,中国建设银行股份有限公司
...,...,...,...,...,...,...,...,...,...
12391,CBT20230705300021,2023-07-05 09:07:54+08:00,220003,100.1510,50000000.0,100132,江苏常熟农村商业银行股份有限公司,102418,国金证券股份有限公司
12392,CBT20230705300052,2023-07-05 09:18:06+08:00,230004,101.6959,30000000.0,102795,中天证券股份有限公司,300921,衡水银行股份有限公司
12393,CBT20230705300034,2023-07-05 09:10:22+08:00,220025,100.6347,30000000.0,101852,江苏海门农村商业银行股份有限公司,100013,平安银行股份有限公司
12394,CBT20230705300027,2023-07-05 09:08:59+08:00,210017,101.4505,10000000.0,103101,三井住友银行（中国）有限公司,101852,江苏海门农村商业银行股份有限公司


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 根据现券的id进行归类计数，升序排列
bond_transactions = df_transaction.groupby('bond_cd').size().sort_values(ascending=False)
print(bond_transactions)


In [ ]:
# 尝试把两个债券的行情画一下
# 230012        550
# 230004        372

# 提取数据
# 通过group将bond_cd为230012的债券筛出来，把行情的图画出来，横轴是时间，纵轴是净值


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# 读取CSV文件并筛选特定债券数据
data = pd.read_csv('ndm_transaction_list_all_20230705.csv')

# print(data['bond_cd'])
filtered_data = data[data['bond_cd'] == 230210]

# print(filtered_data)

# 转换时间戳格式
filtered_data['timeStamp'] = pd.to_datetime(filtered_data['dl_tm'])

# 排序
sorted_data = filtered_data.sort_values(by='timeStamp', ascending=True)

# 创建绘图
plt.figure(figsize=(14, 6))
plt.plot(sorted_data['timeStamp'], sorted_data['net_prc'], marker='o', markersize=3)
plt.xlabel('Time')
plt.ylabel('Net Price')
plt.title('Net Price of Bond 230004 Over Time | Transaction sheet')
plt.xticks(rotation=30)
plt.grid(True)

# 调整 y 轴范围
plt.ylim(min(sorted_data['net_prc']) - 0.5, max(sorted_data['net_prc']) + 1)

plt.show()


In [ ]:
filtered_data = data[data['bond_cd'] == 230004]
sorted_data = filtered_data.sort_values(by='net_prc', ascending=True)
print(sorted_data[['dl_cd', 'net_prc']])


In [ ]:

# AMNT
import pandas as pd
import matplotlib.pyplot as plt

# 读取CSV文件并筛选特定债券数据
data = pd.read_csv('ndm_transaction_list_all_20230705.csv')

# print(data['bond_cd'])
filtered_data = data[data['bond_cd'] == 230004]

# print(filtered_data)

# 转换时间戳格式
filtered_data['timeStamp'] = pd.to_datetime(filtered_data['dl_tm'])

# 排序
sorted_data = filtered_data.sort_values(by='timeStamp', ascending=True)

# 创建绘图
plt.figure(figsize=(14, 6))
plt.bar(sorted_data['timeStamp'], sorted_data['amnt'], width=0.003)
plt.xlabel('Time')
plt.ylabel('amnt')
plt.title('Trade amnt of Bond 230004 Over Time | Transaction sheet')
plt.xticks(rotation=30)
plt.grid(True)

# 调整 y 轴范围
# plt.ylim(min(sorted_data['amnt']) - 0.5, max(sorted_data['amnt']) + 1)

plt.show()


In [ ]:
filtered_data = data[data['bond_cd'] == 230004]
sorted_data = filtered_data.sort_values(by='amnt', ascending=False)
print(sorted_data[['dl_cd', 'amnt']])


In [ ]:
data[data['dl_cd'] == 'CBT20230705307413']['amnt']


In [ ]:

# AMNT
import pandas as pd
import matplotlib.pyplot as plt

# 读取CSV文件并筛选特定债券数据
data = pd.read_csv('ndm_transaction_list_all_20230705.csv')

# print(data['bond_cd'])
filtered_data = data[data['bond_cd'] == 230004]

# print(filtered_data)

# 转换时间戳格式
filtered_data['timeStamp'] = pd.to_datetime(filtered_data['dl_tm'])

# 排序
sorted_data = filtered_data.sort_values(by='timeStamp', ascending=True)

# 创建绘图
plt.figure(figsize=(14, 6))
plt.plot(sorted_data['timeStamp'], sorted_data['yld_to_mrty'], marker='o', markersize=3)
plt.xlabel('Time')
plt.ylabel('amnt')
plt.title('yld_to_mrty of Bond 230004 Over Time | Transaction sheet')
plt.xticks(rotation=30)
plt.grid(True)

# 调整 y 轴范围
plt.ylim(min(sorted_data['yld_to_mrty']) - 0.05, max(sorted_data['yld_to_mrty']) + 0.05)

plt.show()


In [ ]:
filtered_data = data[data['bond_cd'] == 230004]
sorted_data = filtered_data.sort_values(by='yld_to_mrty', ascending=False)
print(sorted_data[['dl_cd', 'yld_to_mrty']])
